## PROMPT
Generate a pipeline for Databricks:
* Get Data information from two web addresses:
   * Space launches: https://api.spacexdata.com/v3/launches
* Aggregate the launches based on launch year
* Send the chosen information to the web address: https://httpbin.org/post
* The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
import requests
import json
import time
from collections import defaultdict

def log_status(message):
    print(f"[STATUS] {message}")

def log_error(message):
    print(f"[ERROR] {message}")

def main():
    start_time = time.time()
    log_status("Pipeline started.")

    # Step 1: Fetch data from the SpaceX API
    spacex_api_url = "https://api.spacexdata.com/v3/launches"
    try:
        log_status(f"Fetching data from {spacex_api_url}...")
        response = requests.get(spacex_api_url)
        response.raise_for_status()
        launches_data = response.json()
        log_status(f"Fetched {len(launches_data)} launches.")
    except requests.RequestException as e:
        log_error(f"Failed to fetch SpaceX data: {e}")
        return

    # Step 2: Aggregate launches by year
    log_status("Aggregating launches by launch year...")
    aggregated_data = defaultdict(int)
    for launch in launches_data:
        year = launch.get("launch_year")
        if year:
            aggregated_data[year] += 1

    result = dict(aggregated_data)
    log_status(f"Aggregated data: {result}")

    # Step 3: Send aggregated data to external endpoint
    post_url = "https://httpbin.org/post"
    try:
        log_status(f"Sending data to {post_url}...")
        post_response = requests.post(post_url, json=result)
        post_response.raise_for_status()
        log_status("Data successfully sent.")
        log_status(f"Response: {post_response.json().get('json')}")
    except requests.RequestException as e:
        log_error(f"Failed to send data: {e}")
        return

    # Step 4: Report execution time
    elapsed_time = time.time() - start_time
    log_status(f"Pipeline completed in {elapsed_time:.2f} seconds.")

# Run the pipeline
main()
